In [2]:
# pip install mne

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
# needs to be < 1.20.0
np.__version__

'1.19.5'

In [8]:
# pip install --upgrade numpy==1.19.5

     |████████████████████████████████| 15.6MB 103kB/s eta 0:00:01
  Found existing installation: numpy 1.21.0
    Uninstalling numpy-1.21.0:
      Successfully uninstalled numpy-1.21.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numba
numba.__version__

'0.44.1'

In [1]:
import mne
file = "JAW_59_18_022019_0910.edf"
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()
info = data.info
channels = data.ch_names

Extracting EDF parameters from /Users/rwu/Desktop/research/Litt/JAW_59_18_022019_0910.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [3]:
info

<Info | 7 non-empty values
 bads: []
 ch_names: Depth_1, Depth_2, Depth_3, Depth_4, Depth_5, Depth_6, Depth_7, ...
 chs: 60 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2019-02-20 00:00:00 UTC
 nchan: 60
 projs: []
 sfreq: 2000.0 Hz
>

In [4]:
channels

['Depth_1',
 'Depth_2',
 'Depth_3',
 'Depth_4',
 'Depth_5',
 'Depth_6',
 'Depth_7',
 'Depth_8',
 'Depth_9',
 'Depth_10',
 'Depth_11',
 'Depth_12',
 'Depth_13',
 'Depth_14',
 'Depth_15',
 'Depth_16',
 'Depth_17',
 'Depth_18',
 'Depth_19',
 'Depth_20',
 'Depth_21',
 'Depth_22',
 'Depth_23',
 'Depth_24',
 'Depth_25',
 'Depth_26',
 'Depth_27',
 'Depth_28',
 'Depth_29',
 'Depth_30',
 'Depth_31',
 'Depth_32',
 'Grid_1',
 'Grid_2',
 'Grid_3',
 'Grid_4',
 'Grid_5',
 'Grid_6',
 'Grid_7',
 'Grid_8',
 'Grid_9',
 'Grid_10',
 'Grid_11',
 'Grid_12',
 'Grid_13',
 'Grid_14',
 'Grid_15',
 'Grid_16',
 'Grid_17',
 'Grid_18',
 'Grid_19',
 'Grid_20',
 'Grid_21',
 'Grid_22',
 'Grid_23',
 'Grid_24',
 'Screw_1',
 'Screw_2',
 'Screw_3',
 'Screw_4']

In [5]:
print(len(raw_data))
print(len(raw_data[0]))
# print(len(raw_data[0][0]))

60
34180000


In [19]:
time = [[1, 2, 3, 1, 2, 1], [5, 4, 3, 2, 2, 4]]
print(np.fft.rfft(time))
print(np.fft.rfft(time[0]))
print(np.fft.rfft(time[1]))
print(np.fft.rfft(time, axis = 1))
print(np.absolute(np.fft.rfft(time)))

[[10. +0.j         -1. -1.73205081j -2. +0.j          2. +0.j        ]
 [20. +0.j          4.5-0.8660254j   0.5+0.8660254j   0. +0.j        ]]
[10.+0.j         -1.-1.73205081j -2.+0.j          2.+0.j        ]
[20. +0.j         4.5-0.8660254j  0.5+0.8660254j  0. +0.j       ]
[[10. +0.j         -1. -1.73205081j -2. +0.j          2. +0.j        ]
 [20. +0.j          4.5-0.8660254j   0.5+0.8660254j   0. +0.j        ]]
[[10.          2.          2.          2.        ]
 [20.          4.58257569  1.          0.        ]]


In [25]:
a = [1, 2, 3, 2, 5]
a[:3]
abs(np.diff(a))

array([1, 1, 1, 3])

In [2]:
import numpy as np
from sklearn.preprocessing import scale
from scipy.signal import resample

class Seizure(object):
    
    def __init__(self, input_data):
        
        self.data = input_data
        self.num_chan = len(input_data)
        self.time = len(input_data[0])
        
    def fft_bychan(self, chan_num, time_int):
        
        return [np.absolute(np.fft.rfft(self.data[chan_num][i*time_int:i*time_int+time_int])[:150]) for i in range(self.time//time_int)]
    
    def line_len_bychan(self, chan_num, time_int):
        
        return [sum(abs(np.diff(self.data[chan_num][i*time_int:i*time_int+time_int]))) for i in range(self.time//time_int)]
    
    def freq_corr(self, chan_num, time_int):
        
        fft = self.fft_bychan(chan_num, time_int)
        scaled = scale(fft, axis = 0)
        corr_mtx = np.corrcoef(scaled)
        eigenval = np.absolute(np.linalg.eig(corr_mtx)[0])
        eigenval.sort()
        return corr_mtx, eigenval
        
    def time_corr(self, chan_num, time_int):
        
        ptt = [self.data[chan_num][i*time_int:i*time_int+time_int] for i in range(self.time//time_int)]
        scaled = scale(ptt, axis = 0)
        corr_mtx = np.corrcoef(scaled)
        eigenval = np.absolute(np.linalg.eig(corr_mtx)[0])
        eigenval.sort()
        return corr_mtx, eigenval        
        

In [8]:
a = Seizure(raw_data)
b = a.fft_bychan(20, 2000)
print(len(b))
print(len(b[0]))
b

17090
150


[array([9.16165654e-05, 1.49083961e-03, 3.61042719e-03, 1.22195713e-03,
        8.07595357e-04, 1.07632303e-03, 3.27091601e-04, 7.02308959e-04,
        4.89746826e-04, 4.44959297e-04, 4.65188370e-04, 5.66426762e-04,
        1.62121418e-04, 3.48114794e-04, 3.08882933e-04, 1.90467882e-04,
        2.77177249e-04, 1.97680234e-04, 4.88401518e-04, 2.56795209e-04,
        6.59668402e-04, 2.60385097e-04, 5.53171248e-04, 1.53066378e-04,
        1.81335822e-04, 2.59246348e-04, 3.89205964e-04, 1.56265923e-04,
        1.64461425e-04, 1.03797819e-04, 9.08513162e-05, 2.23206659e-04,
        1.70765974e-04, 1.99192296e-04, 1.19770920e-04, 2.13742122e-05,
        4.57635477e-05, 1.39514080e-04, 1.02114894e-04, 1.64994174e-04,
        2.99739531e-04, 1.13627868e-04, 1.97649914e-04, 8.01425080e-05,
        1.81272936e-04, 1.94293387e-04, 2.79833929e-04, 7.64013060e-05,
        1.04275545e-04, 1.79014669e-04, 1.59113405e-04, 1.53167618e-04,
        1.24816609e-04, 1.91624702e-04, 6.34810112e-05, 1.459803

In [13]:
a = [np.array([[1, 2], [3, 4]]), np.array([[5, 2], [3, 4]])]
sum(a)
sum(np.array([1, 2, 3, 4]))

10

In [3]:
onset = [(41, 18), (122, 57), (613, 26), (646, 10), (670, 25), (849, 16), \
        (898, 24), (1015, 44), (1244, 22), (1278, 58), (1356, 19), \
        (1799, 41), (2069, 30), (2170, 13), (2205, 41), (2257, 53), \
        (2363, 23), (12761, 11)]
onset_time = []
for a, b in onset:
    onset_time.append(list(range(a, a+b+1)))
print(onset_time)

[[41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179], [613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639], [646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656], [670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695], [849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865], [898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922], [1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029

In [22]:
# fft analysis
import scipy.stats as ss

a = Seizure(raw_data)
chan_sum = sum([np.array(a.fft_bychan(i, 2000)) for i in range(60)])
# expect 17090
print(len(chan_sum))
# expect 150
print(len(chan_sum[0]))
fft_sum = [sum(i) for i in chan_sum]
# expect 17090
print(len(fft_sum))
res = ss.rankdata(fft_sum)
rank_res = [[res[j] for j in i] for i in onset_time]
print(rank_res)
print([sum(i)/len(i) for i in rank_res])

17090
150
17090
[[12325.0, 12982.0, 10233.0, 10072.0, 10429.0, 11218.0, 14196.0, 12560.0, 10618.0, 6415.0, 7443.0, 5309.0, 6491.0, 7712.0, 8075.0, 6051.0, 3787.0, 4368.0, 6999.0], [1072.0, 2176.0, 2279.0, 8895.0, 9749.0, 10084.0, 6725.0, 5170.0, 3423.0, 7529.0, 4446.0, 5132.0, 5814.0, 8054.0, 7451.0, 7199.0, 9054.0, 6392.0, 7163.0, 7138.0, 7078.0, 8956.0, 12024.0, 12861.0, 10763.0, 11536.0, 14272.0, 13149.0, 12188.0, 14963.0, 15414.0, 15500.0, 16387.0, 15865.0, 16633.0, 16189.0, 16952.0, 16084.0, 15223.0, 16167.0, 15833.0, 17005.0, 17088.0, 16032.0, 16007.0, 14987.0, 13990.0, 10604.0, 10004.0, 13970.0, 14789.0, 14081.0, 12979.0, 12385.0, 15203.0, 13178.0, 13081.0, 11512.0], [5245.0, 6928.0, 7938.0, 15443.0, 15551.0, 13864.0, 10101.0, 7469.0, 10432.0, 8528.0, 10522.0, 11792.0, 13646.0, 14229.0, 13981.0, 14175.0, 15321.0, 14842.0, 14722.0, 14717.0, 16224.0, 15800.0, 16485.0, 16437.0, 15510.0, 15222.0, 15016.0], [4058.0, 10153.0, 9040.0, 14110.0, 15827.0, 15793.0, 15487.0, 15358.0, 15070.

In [4]:
import scipy.stats as ss

a = Seizure(raw_data)
chan_sum_line = sum([np.array(a.line_len_bychan(i, 2000)) for i in range(60)])
# expect 17090
print(len(chan_sum_line))
res_line = ss.rankdata(chan_sum_line)
rank_res_line = [[res_line[j] for j in i] for i in onset_time]
print(rank_res_line)
print([sum(i)/len(i) for i in rank_res_line])
print([max(i) for i in rank_res_line])

17090
[[11504.0, 11467.0, 11931.0, 12893.0, 13481.0, 14352.0, 15889.0, 13103.0, 12160.0, 9464.0, 10762.0, 6719.0, 8582.0, 11350.0, 10362.0, 9595.0, 8422.0, 9300.0, 8708.0], [642.0, 1436.0, 2310.0, 5279.0, 8600.0, 8778.0, 5976.0, 4543.0, 3224.0, 6227.0, 4535.0, 3983.0, 5720.0, 8621.0, 7420.0, 6774.0, 7052.0, 6890.0, 6013.0, 8565.0, 9126.0, 10824.0, 12397.0, 12624.0, 9676.0, 12795.0, 14501.0, 14375.0, 12475.0, 15907.0, 14606.0, 14190.0, 16522.0, 15429.0, 17009.0, 16893.0, 17066.0, 16333.0, 12796.0, 16759.0, 16743.0, 17051.0, 17087.0, 12565.0, 16377.0, 15391.0, 10953.0, 8765.0, 11281.0, 14551.0, 11368.0, 11967.0, 9743.0, 14296.0, 16360.0, 11904.0, 14740.0, 7594.0], [5213.0, 12130.0, 14923.0, 16274.0, 15711.0, 13819.0, 12723.0, 11307.0, 13488.0, 11393.0, 12871.0, 13643.0, 15720.0, 15680.0, 15297.0, 15564.0, 16439.0, 15050.0, 15639.0, 16041.0, 16641.0, 16733.0, 16448.0, 15266.0, 13911.0, 11489.0, 8654.0], [6393.0, 9400.0, 10411.0, 14597.0, 16389.0, 16267.0, 15998.0, 15769.0, 15531.0, 14619.

In [7]:
for i in onset_time:
    print([chan_sum_line[j] for j in i])

[0.048535985137485596, 0.048392713705862296, 0.05045981246375895, 0.05529493697927749, 0.058817352824487516, 0.06482390362254657, 0.0871224715414892, 0.056515667439801937, 0.05148718375194517, 0.043041281624805056, 0.0461365352336188, 0.03848209173863911, 0.041321542252876035, 0.04798471861934223, 0.04505238578447791, 0.04329648197881981, 0.04107471999267517, 0.04267318796349944, 0.04153931241798106]
[0.02815967741935455, 0.029939389477217637, 0.03135546815393504, 0.0365507904293952, 0.041339323099459466, 0.041674386577959174, 0.037616194647053104, 0.03516912823877654, 0.03284948118533864, 0.037924436170537125, 0.035150383007293555, 0.034156825464644026, 0.03728926816614247, 0.04138356425672138, 0.039480652027954595, 0.03854103976561753, 0.03891793343913047, 0.038704563280129084, 0.03766109881893335, 0.04129387646107332, 0.04233083132419779, 0.04630469985045893, 0.05265011139255929, 0.05379055681020488, 0.043455123294777964, 0.05477874996185151, 0.06606006424146214, 0.06501672032837881